## Deployment of Mixed Model to Azure ML using MLFlow API

In [1]:
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config()

# mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())


In [60]:
#Save the python flavor of the model together with environment spec
import mlflow.pyfunc
from sys import version_info

import glob
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np
from tensorflow.keras.models import Model
import tensorflow as tf
import sklearn
# Construct and save the model
PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                                  minor=version_info.minor,
                                                  micro=version_info.micro)

conda_env = {
    'channels': ['defaults'],
    'dependencies': [
      'python={}'.format(PYTHON_VERSION),
      'pip',
      {
        'pip': [
          'mlflow',
            'opencv-python-headless',
            'pillow',
            'azureml-mlflow',
          'tensorflow=={}'.format(tf.__version__),
          'scikit-learn=={}'.format(sklearn.__version__),
        ],
      },
    ],
    'name': 'tf_env'
}


model_path = "mixed_model"
mlflow.pyfunc.save_model(path=model_path, data_path ='output',loader_module ='mixedmodel.model_loader', code_path=['mixedmodel'], conda_env=conda_env)



### Preparing data to test service


In [46]:
#Testing the deployed service

from io import BytesIO
import pandas as pd
import json
import base64
ENCODING = 'utf-8'
test_image_paths =['dataset/images/airplane/image_0001.jpg','dataset/images/airplane/image_0002.jpg','dataset/images/airplane/image_0003.jpg']
additional_feature_file ='output/sample_feature.json'
feature_file_df = pd.read_json(additional_feature_file)
base64_string_list=[]
for test_image_path in test_image_paths:
    with open(test_image_path,"rb") as img:
        image_bytes = BytesIO(img.read())
    encoded_image =base64.b64encode(image_bytes.getvalue())
    base64_string = encoded_image.decode(ENCODING)
    base64_string= "b'{0}'".format(base64_string)
    base64_string_list.append(base64_string)
image_request = {"image": base64_string_list, "additional_feature":[feature_file_df.iloc[i].to_json() for i in range(feature_file_df.shape[0])]}
image_request= {"data": image_request}
# payload.append(image_request)

body = json.dumps(image_request)

In [57]:
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
import ast
import joblib

from mlflow.pyfunc.scoring_server import parse_json_input, _get_jsonable_obj




restored_model = joblib.load("output/rf_sample.pkl")
base_model = VGG19(weights='imagenet')
model= Model(base_model.input, outputs = base_model.get_layer("fc2").output)

def base64ToImg(base64ImgString):

    if base64ImgString.startswith('b\''):

        base64ImgString = base64ImgString[2:-1]

    base64Img   =  base64ImgString.encode('utf-8')

    decoded_img = base64.b64decode(base64Img)

    img_buffer  = BytesIO(decoded_img)

    img = Image.open(img_buffer)
    return img

features_list = []
addional_feature_list=[]

for row_idx in range(imagePathDF.shape[0]):
    base64_string = imagePathDF.iloc[row_idx]['image']
    img = base64ToImg(base64_string)
    image= img.resize((224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    x = preprocess_input(image)
    features = model.predict(x)
    features_list.append(features)
    addional_feature = imagePathDF.iloc[row_idx]["additional_feature"]
    addional_feature_list.append(ast.literal_eval(addional_feature))
new_features = np.concatenate([pd.DataFrame(addional_feature_list).values, np.concatenate(features_list)], axis=1)

In [61]:
import json
# data = json.loads(body)['data']
imagePathDF = parse_json_input(json_input=body, orient="split")

In [62]:
#Reload model locally and score 

import mlflow
# model_path = "model"
loaded_model = mlflow.pyfunc.load_model(model_path)

pred_output = loaded_model.predict(imagePathDF)
print(pred_output)

[0, 0, 0]


### Deploy model to ACI

In [65]:
from azureml.core.webservice import AciWebservice, Webservice

# Set the model path to the model folder created by your run

# Configure 
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=4, 
                                                tags={'method' : 'keras/tf'}, 
                                                description='aml-mlflow model',
                                                location='westus2')


In [67]:
import mlflow.azureml
(webservice,model) = mlflow.azureml.deploy( model_uri=model_path,
                      workspace=ws,
                      model_name='aml_mflow_mixedmodel', 
                      service_name='aml-mlflow-mixedmodel-v1', 
                      deployment_config=aci_config, 
                      tags=None, mlflow_home=None, synchronous=True)

webservice.wait_for_deployment(show_output=True)


Registering model aml_mflow_mixedmodel


2021/01/24 20:37:04 INFO mlflow.azureml: Registered an Azure Model with name: `aml_mflow_mixedmodel` and version: `2`
2021/01/24 20:37:07 INFO mlflow.azureml: Deploying an Azure Webservice with name: `aml-mlflow-mixedmodel-v1`


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"


### Deploy model to AKS

In [91]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (can also provide parameters to customize)
# prov_config = AksCompute.provisioning_configuration()

prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_D11_v2",
                                                   agent_count = 3,
                                                   location = "westus2")

aks_name = 'aks-mlflowd11'

# Create the cluster
aks_target = ComputeTarget.create(workspace=ws, 
                                  name=aks_name, 
                                  provisioning_configuration=prov_config)

aks_target.wait_for_completion(show_output = True)

print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)


Creating.......................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None


In [94]:
from azureml.core.webservice import Webservice, AksWebservice

# Set the web service configuration (using default here with app insights)
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True, cpu_cores = 1, memory_gb = 7, compute_target_name='aks-mlflowd11',autoscale_enabled=True, 
                                                autoscale_target_utilization=30,
                                                autoscale_min_replicas=3)



In [95]:
import mlflow.azureml
# Webservice creation using single command
from azureml.core.webservice import AksWebservice, Webservice

# set the model path 
model_path = "model"

(webservice,model) = mlflow.azureml.deploy( model_uri=model_path,
                      workspace=ws,
                      model_name='aml_mflow_mixedmodel', 
                      service_name='aml-mlflow-mixedmodel-aks-d11', 
                      deployment_config=aks_config, 
                      tags=None, mlflow_home=None, synchronous=True)


webservice.wait_for_deployment()


2021-01-20 10:43:44,645 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /azureml/LocalUpload/210120T104342-7317119f/model/code/pyimagesearch/__init__.py?sv=2019-02-02&sr=b&sig=wBIk36kjFdB3hEkyt6ddljK7momc2kD6uy8sYdoEp0M%3D&st=2021-01-20T18%3A33%3A45Z&se=2021-01-21T18%3A43%3A45Z&sp=rcw&comp=blocklist&timeout=30
2021-01-20 10:43:44,657 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /azureml/LocalUpload/210120T104342-7317119f/model/code/pyimagesearch/__pycache__/__init__.cpython-37.pyc?sv=2019-02-02&sr=b&sig=cOuOckQOqqN1Lvaz8E3vvIFE39cMsmJ4W1K2cG1jegY%3D&st=

Registering model object_detection_model


2021/01/20 10:44:08 INFO mlflow.azureml: Registered an Azure Model with name: `object_detection_model` and version: `27`
2021/01/20 10:44:16 INFO mlflow.azureml: Deploying an Azure Webservice with name: `aml-mlflow-test-aks-d11`


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................
Succeeded
AKS service creation operation finished, operation "Succeeded"
AKS service creation operation finished, operation "Succeeded"


In [70]:
#Score model with SDK object
response = webservice.run(body)
print(response)

[0, 0, 0]


In [80]:

#another way to call http request directly 
import requests
uri = webservice.scoring_uri
api_key = '' # Replace this with the API key for the web service

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

# uri ='http://9c729eb8-c2d1-4f24-9114-fbd35d040d7e.westus2.azurecontainer.io/score'
response = requests.post(url=uri, data=body,headers=headers)
print(response.text)


[[0.12486386299133301, 0.1821191906929016, 0.8843520283699036, 0.834100604057312, "airplane"], [0.1420917510986328, 0.18152016401290894, 0.8583660125732422, 0.8296916484832764, "airplane"]]


In [71]:
webservice.scoring_uri

'http://486cc62b-8960-45f2-aa93-76cd3d844d56.westus2.azurecontainer.io/score'